In [52]:
import time
import json
import mistralai
import pandas as pd
from dotenv import load_dotenv

In [53]:
load_dotenv()

True

In [54]:
client = mistralai.Mistral(api_key=os.getenv('MISTRAL_API_KEY'))

In [55]:
df = pd.read_excel("../data/raw/AI_dataset.xlsx")
df = df[df['Описание'].notna()].copy()
df.head()

,Описание,Решение
0,"№ 57 маршрут, на остановке Нурлы Жол в направл...",NaN
1,14 м/а с 6:00-6:31 нету автобуса с Нурлы жол в...,NaN
2,06:00-06:35 аралығында «Ақ-Жайық» ТД аялдамасы...,NaN
3,"№2,13,32, 49, 50,73 Балбала аялдамасы Көктал ...",NaN
4,"№ 18 бағдар , мем/нөмір 862 . Медицинский цен...",NaN


In [59]:
def label_example(review: str):
    time.sleep(1)  # rate limit
    response = client.chat.complete(
        model="mistral-large-latest",
        temperature=0,
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": """Your goal is to make sentiment analysis and label reviews about public transport
                \rThey can be either positive or negative.
                \rProvide classification result with in JSON format:
                \r{"label": <the label>}
                ]r"<the label>" has to be INT type and 1 if review is positive and 0 if review is negative
                """
            },
            {
                "role": "user",
                "content": f"Here is the review for you to classify: {review}"
            }
        ]
    )
    label_json = response.choices[0].message.content
    return json.loads(label_json)['label']


# test
label_example("всегда вовремя")

1

In [62]:
df['label'] = df['Описание'].apply(label_example)

In [63]:
df.to_csv("../data/interim/labeled_reviews.csv", index=False)